## 딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회

**※주의** : 반드시 본 파일을 이용하여 제출을 수행해야 하며, 파일의 이름은 `task.ipynb`로 유지되어야 합니다.

* #### 추론 실행 환경
    * `python 3.9` 환경
    * `CUDA 10.2`, `CUDA 11.8`, `CUDA 12.6`를 지원합니다.
    * 각 CUDA 환경에 미리 설치돼있는 torch 버전은 다음 표를 참고하세요.

<table>
  <thead>
    <tr>
      <th align="center">Python</th>
      <th align="center">CUDA</th>
      <th align="center">torch</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center" style="vertical-align: middle;">3.8</td>
      <td align="center">10.2</td>
      <td align="center">1.6.0</td>
    </tr>
    <tr>
      <td align="center" style="vertical-align: middle;">3.9</td>
      <td align="center">11.8</td>
      <td align="center">1.8.0</td>
    </tr>
    <tr>
      <td align="center">3.10</td>
      <td align="center">12.6</td>
      <td align="center">2.7.1</td>
    </tr>
  </tbody>
</table>

* #### CUDA 버전 관련 안내사항  
  - 이번 경진대회는 3개의 CUDA 버전을 지원합니다.  
  - 참가자는 자신의 모델의 라이브러리 의존성에 맞는 CUDA 환경을 선택하여 모델을 제출하면 됩니다.   
  - 각 CUDA 환경에는 기본적으로 torch가 설치되어 있으나, 참가자는 제출하는 CUDA 버전과 호환되는 torch, 필요한 버전의 라이브러리를 `!pip install` 하여 사용하여도 무관합니다.

* #### `task.ipynb` 작성 규칙
코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하세요.   
1. **제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치**
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 그 외로, 모델 추론에 필요한 라이브러리를 직접 설치합니다.
2. **추론용 코드 작성**
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. **aif.submit() 함수를 호출하여 최종 결과를 제출**
    - **마이 페이지-활동히스토리**에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.
    - **※주의** : 제출하고자 하는 CUDA 환경에 맞는 key를 입력하여야 합니다.

<table>
  <thead>
    <tr>
      <th align="left">Competition 이름</th>
      <th align="center">CUDA</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회</td>
      <td align="center">11.8</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 12.6</td>
      <td align="center">12.6</td>
    </tr>
    <tr>
      <td align="left">딥페이크 범죄 대응을 위한 AI 탐지 모델 경진대회 CUDA 10.2</td>
      <td align="center">10.2</td>
    </tr>
  </tbody>
</table>

------

#### 1. 제출용 aifactory 라이브러리 설치
※ 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 33.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=ef7c35d9a6ec4d2645294b65fcda85ad2c7eb33b2eda7e01c07690050db14aec
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: IPython
    Found existing installation: ipython 8.21.0
    Uninstalling ipython-8.21.0:
      Successfully uninstalled ipython-8.21.0

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


* 자신의 모델 추론 실행에 필요한 추가 라이브러리 설치

In [2]:
!apt-get update -y && apt-get install -y cmake build-essential

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2123 kB]
Get:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1289 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [38.5 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/mu

In [3]:
!pip install --index-url https://download.pytorch.org/whl/cu126 torch==2.7.1 torchvision==0.22.1

!pip install transformers==4.30 datasets==4.4.0 tqdm==4.67.1
!pip install opencv-python-headless==4.10.0.82 numpy==1.26.4 scikit-learn==1.3.2 scipy==1.11.4
!pip install pandas Pillow
!pip install -U dlib

Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 822.1/822.1 MB 8.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 99.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 16.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 95.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 11.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 28.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 58.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 32.

-----

#### 2. 추론용 코드 작성

##### 추론 환경의 기본 경로 구조

- 평가 데이터셋 경로: `./data/`
   - 채점에 사용될 테스트 데이터셋은 `./data/` 디렉토리 안에 포함되어 있습니다.
   - 해당 디렉토리에는 이미지(JPG, PNG)와 동영상(MP4) 파일이 별도의 하위 폴더 없이 혼합되어 있습니다.
```bash
/aif/
└── data/
    ├── {이미지 데이터1}.jpg
    ├── {이미지 데이터2}.png
    ├── {동영상 데이터1}.mp4
    ├── {이미지 데이터3}.png
    ├── {동영상 데이터2}.mp4
    ...
```

- 모델 및 자원 경로: 예시 : `./model/`
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치해야하 하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)

* 제출 파일은 `submission.csv`로 저장돼야 합니다.
  * submission.csv는 *filename*과 *label* 컬럼으로 구성돼야 합니다.
  * filename은 추론한 파일의 이름(확장자 포함), label은 추론 결과입니다. (real:0, fake:1)
  * filename은 *string*, label은 *int* 자료형이어야 합니다.
  * 추론하는 데이터의 순서는 무작위로 섞여도 상관 없습니다.

<table>
  <thead>
    <tr>
      <th align="center">filename</th>
      <th align="center">label</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td align="center">{이미지 데이터1}.jpg</td>
      <td align="center">0</td>
    </tr>
    <tr>
      <td align="center">{동영상 데이터1}.mp4</td>
      <td align="center">1</td>
    </tr>
    <tr>
      <td colspan="2" align="center">...</td>
    </tr>
  </tbody>
</table>

**※ 주의 사항**

* argparse 사용시 `args, _ = parser.parse_known_args()`로 인자를 지정하세요.   
   - `args = parser.parse_args()`는 jupyter에서 오류가 발생합니다.
* return 할 결과물과 양식에 유의하세요.

In [ ]:
# infer.py
import os, json, csv, cv2, dlib, torch, numpy as np, torch.nn.functional as F
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from transformers import AutoImageProcessor, AutoFeatureExtractor

# ✅ 우리 모델 코드 임포트
from src import ClipBackbone
from src import UnifiedAdapterModel  

MODEL_DIR = "./model/clip_base"
TEST_DIR  = Path("./data")
SUBMIT_CSV = Path("submission.csv")
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
VIDEO_EXTS = {".mp4", ".avi"}

def load_processor(model_dir):
    try:
        return AutoImageProcessor.from_pretrained(model_dir)
    except Exception:
        return AutoFeatureExtractor.from_pretrained(model_dir)

def get_boundingbox(face, width, height):
    x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * 1.3)
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    x1 = max(int(cx - size_bb // 2), 0)
    y1 = max(int(cy - size_bb // 2), 0)
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)
    return x1, y1, size_bb

detector = dlib.get_frontal_face_detector()

def detect_and_crop_face_optimized(image: Image.Image, target_size=(224, 224), resize_for_detection=640):
    if image.mode != "RGB":
        image = image.convert("RGB")
    np_img = np.array(image)
    H, W, _ = np_img.shape
    if W > resize_for_detection:
        scale = resize_for_detection / float(W)
        resized = cv2.resize(np_img, (resize_for_detection, int(H * scale)), interpolation=cv2.INTER_AREA)
    else:
        scale, resized = 1.0, np_img
    faces = detector(resized, 1)
    if not faces:
        return None
    face = max(faces, key=lambda r: r.width() * r.height())
    face_scaled = dlib.rectangle(
        left=int(face.left()/scale), top=int(face.top()/scale),
        right=int(face.right()/scale), bottom=int(face.bottom()/scale)
    )
    x, y, s = get_boundingbox(face_scaled, W, H)
    crop = np_img[y:y+s, x:x+s]
    if crop.size == 0: return None
    return Image.fromarray(crop).resize(target_size, Image.BICUBIC)

def process_single_file(path: Path):
    faces = []
    ext = path.suffix.lower()
    try:
        if ext in IMAGE_EXTS:
            img = Image.open(path)
            f = detect_and_crop_face_optimized(img)
            if f: 
                faces = [f] * 12
        elif ext in VIDEO_EXTS:
            cap = cv2.VideoCapture(str(path))
            total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if total > 0:
                idxs = np.linspace(0, total - 1, 12, dtype=int)
                for i in idxs:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                    ok, frame = cap.read()
                    if not ok: continue
                    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                    f = detect_and_crop_face_optimized(img)
                    if f: faces.append(f)
            cap.release()
    except Exception as e:
        return path.name, [], str(e)
    return path.name, faces, None

def load_custom_model(model_dir: str, device: torch.device):
    with open(os.path.join(model_dir, "custom_config.json"), "r") as f:
        cfg = json.load(f)

    clip_name     = os.path.join(model_dir, "clip_backbone")   
    num_frames    = int(cfg.get("num_frames", 12))
    adapter_type  = cfg.get("adapter_type", "tconv")
    temporal_pool = cfg.get("temporal_pool", "mean")
    head_hidden   = int(cfg.get("head_hidden", 1024))
    num_classes   = int(cfg["num_classes"])
    id2label      = cfg.get("id2label")
    label2id      = cfg.get("label2id")

    backbone = ClipBackbone(
        model_name=clip_name,      
        dtype="fp32",
        freeze_backbone=True,
    )
    model = UnifiedAdapterModel(
        backbone=backbone,
        num_frames=num_frames,
        adapter_type=adapter_type,
        temporal_pool=temporal_pool,
        head_hidden=head_hidden,
        num_classes=num_classes,
        id2label=id2label, label2id=label2id,
    )
    state_path = os.path.join(model_dir, "model.bin") 
    state = torch.load(state_path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.to(device).eval()
    return model, id2label

if __name__ == "__main__":
    torch.backends.cudnn.benchmark = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    processor = load_processor(MODEL_DIR)
    model, id2label = load_custom_model(MODEL_DIR, device)
    print("model loaded on:", next(model.parameters()).device)

    with open(SUBMIT_CSV, "w", newline="") as f:
        csv.writer(f).writerow(["filename", "label"])

    files = [p for p in sorted(TEST_DIR.iterdir()) if p.is_file()]
    print("Test files:", len(files))

    results = {}
    from multiprocessing import Pool, cpu_count
    nproc = min(max(1, os.cpu_count() - 1), 8)

    with Pool(processes=nproc) as pool, tqdm(total=len(files), desc="Preprocessing files") as pbar:
        for fname, face_images, err in pool.imap_unordered(process_single_file, files):
            if err:
                print(f"[WARN] {fname}: {err}")

            if not face_images:
                # 얼굴 못 찾으면 기본값(예: 0=real)로
                results[fname] = 0
                pbar.update(1); continue

            with torch.no_grad():
                batch = 8
                probs_all = []
                for i in range(0, len(face_images), batch):
                    sub = face_images[i:i+batch]
                    enc = processor(images=sub, return_tensors="pt")
                    pixel_values = enc["pixel_values"].to(device)
                    logits = model(pixel_values) if hasattr(model, '__call__') else model.forward(pixel_values=pixel_values)
                    probs = F.softmax(logits, dim=1)
                    probs_all.append(probs.detach())
                probs_mean = torch.cat(probs_all, dim=0).mean(dim=0)
                pred = int(torch.argmax(probs_mean).item())
                results[fname] = pred

            pbar.update(1)

    print("Writing results...")
    with open(SUBMIT_CSV, "a", newline="") as f:
        w = csv.writer(f)
        for p in files:
            w.writerow([p.name, int(results.get(p.name, 0))])

    print("Done.")

/opt/anaconda3/envs/deepfake/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


model loaded on: cpu
Test files: 12


Preprocessing files:   0%|          | 0/12 [00:00<?, ?it/s]Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-5:
Process SpawnPoolWorker-6:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/deepfake/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/deepfake/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/deepfake/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/deepfake/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/deepfake/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/deepfake/lib/python3.10/multiprocessin

KeyboardInterrupt: 

----

#### 3. `aif.submit()` 함수를 호출하여 최종 결과를 제출

**※주의** : task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9197/baseline) 탭에 기재된 가이드라인을 따라 task 별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 task에 맞는 자신의 key를 사용해야 합니다.
-  **NOTE** : 이번 경진대회에서는 3개의 CUDA 버전을 지원하며, 각 CUDA 버전에 따라 task key가 상이합니다. 함수를 실행하기 전에 현재 key가 제출하고자 하는 CUDA 환경에 대한 key인지 반드시 확인하세요.

In [ ]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="sdxl_fix",
    key="d4125af8-8c7e-4680-b61d-57c8df53d8a7"
)
#-----------------------------------------------------#
print(time.time() - t)

file : task
jupyter notebook
제출 완료
650.9502041339874
